In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
# Configuração
base_path = "../data/processed/"
os.makedirs("../models", exist_ok=True)

df = pd.read_csv(f"{base_path}df_modelagem_v3.csv", index_col="data", parse_dates=True)

targets = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot", 
    "Rural_PJ": "inad_rd_pj_cr_rur_tot" 
}

# --- CONFIGURAÇÃO DOS MODELOS ---
# Ridge com alpha baixo para permitir que a Selic faça preço
modelos_config = {
    "Ridge": Ridge(alpha=1.0), 
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)
}

# --- A MUDANÇA RADICAL ---
# Removemos o Lag_1 das features. O modelo fica "cego" para o passado e olha só economia.
all_targets = list(targets.values())
cols_to_drop_base = [t for t in all_targets if t in df.columns] + \
                    ["presidente", "target_lag_1"] 

print(f"🔥 Iniciando Treino de Sensibilidade Pura (Delta Macro)...")

for segmento, target_col in targets.items():
    if target_col not in df.columns: continue
    
    print(f"--- Processando: {segmento} ---")
    
    df_temp = df.copy()
    
    # Target é o DELTA (Diferença)
    df_temp["target_diff"] = df_temp[target_col] - df_temp[target_col].shift(1)
    
    # Lag apenas para gerar o Delta, depois jogamos fora
    df_temp["target_lag_1"] = df_temp[target_col].shift(1) 
    df_temp = df_temp.dropna()
    
    y = df_temp["target_diff"] # Prever a Variação
    
    # X contém APENAS Macroeconomia + Sazonalidade (Sem lag de inadimplência)
    X = df_temp.drop(columns=cols_to_drop_base + [target_col, "target_diff"])
    
    # Padronização
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    joblib.dump(scaler, f"../models/scaler_{segmento}.pkl")
    pd.DataFrame(columns=X.columns).to_csv(f"../models/columns_{segmento}.csv", index=False)
    # Salvamos o último X (que agora só tem macroeconomia)
    X.iloc[-1].to_csv(f"../models/last_values_{segmento}.csv")
    
    for nome, modelo in modelos_config.items():
        from sklearn.base import clone
        mdl = clone(modelo)
        mdl.fit(X_scaled, y)
        joblib.dump(mdl, f"../models/model_{segmento}_{nome}.pkl")
        
        # Check rápido de sensibilidade (Só para o Ridge)
        if nome == "Ridge":
            try:
                coefs = pd.DataFrame({'var': X.columns, 'coef': mdl.coef_})
                selic_impact = coefs.loc[coefs['var'].str.contains('selic'), 'coef'].sum()
                print(f"   > Sensibilidade Selic (Ridge): {selic_impact:.4f}")
            except: pass

print("\n✅ Treino Concluído! Agora os modelos dependem 100% da Economia.")

🔥 Iniciando Treino de Sensibilidade Pura (Delta Macro)...
--- Processando: PF ---
   > Sensibilidade Selic (Ridge): -0.0826
--- Processando: PJ ---
   > Sensibilidade Selic (Ridge): -0.1334
--- Processando: Rural_PF ---
   > Sensibilidade Selic (Ridge): -0.0051
--- Processando: Rural_PJ ---
   > Sensibilidade Selic (Ridge): -0.3760

✅ Treino Concluído! Agora os modelos dependem 100% da Economia.
